# CSE 144 Fall 2023 Final Project



In [1]:
import torch
import torchvision
from torchvision import models, transforms, datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import os
import pandas as pd
from PIL import Image


#### Step-1: Data Loading and Preprocessing

In [19]:
# Step-1: Data Loading and Preprocessing
class CustomDataset(Dataset):
    def __init__(self, directory, transform=None, train=True):
        self.directory = directory
        self.transform = transform
        self.train = train  # Flag to indicate if it's training data
        self.images = []
        self.labels = []

        if train:
            # Handle train dataset with labels
            for label_dir in os.listdir(directory):
                label_path = os.path.join(directory, label_dir)
                if os.path.isdir(label_path):
                    for image_file in os.listdir(label_path):
                        image_path = os.path.join(label_path, image_file)
                        self.images.append(image_path)
                        self.labels.append(int(label_dir))
        else:
            # Handle test dataset without labels
            for image_file in os.listdir(directory):
                image_path = os.path.join(directory, image_file)
                self.images.append(image_path)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        if self.train:
            label = self.labels[idx]
            return image, label
        else:
            return image


# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Assuming the train and test directories are in the same directory as the script
train_dir = 'train'
test_dir = 'test'

# Transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Add horizontal flip
    transforms.RandomApply([transforms.Grayscale(num_output_channels=3)], p=0.2),  # Convert to grayscale with a probability of 0.2
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2),  # Add color jitter
    transforms.RandomRotation(15),  # Randomly rotate by +/- 15 degrees
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load Datasets with respective transformations
train_data = CustomDataset(train_dir, transform=train_transforms, train=True)
test_data = CustomDataset(test_dir, transform=test_transforms, train=False)

# Load Datasets

train_size = int(0.9 * len(train_data))
val_size = len(train_data) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(train_data, [train_size, val_size])

print(len(train_dataset))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=0)


971


#### Step-2: Model Setup

In [3]:
# Step-2: Model Setup

# Load a pre-trained model, for example, ResNet-50
model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 100)  # Assuming 100 classes from the combined datasets



C:\Users\timmy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\timmy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
print(torch.version.cuda)
device = torch.device("cpu")

None


#### Step-3: Loss function and Optimizer

In [5]:
# Step-3: Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.005, momentum=0.9)

#### Step-4: Training Loop & Testing Loop

In [6]:
# Step-4: Training Loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # Reset the gradients
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

train_model(model, train_loader, criterion, optimizer)


Epoch 1/20, Loss: 4.5576, Acc: 0.0237
Epoch 2/20, Loss: 4.1309, Acc: 0.0669
Epoch 3/20, Loss: 3.6585, Acc: 0.1617
Epoch 4/20, Loss: 3.3267, Acc: 0.2410
Epoch 5/20, Loss: 3.0657, Acc: 0.3090
Epoch 6/20, Loss: 2.8562, Acc: 0.3481
Epoch 7/20, Loss: 2.6198, Acc: 0.3975
Epoch 8/20, Loss: 2.5007, Acc: 0.4325
Epoch 9/20, Loss: 2.3596, Acc: 0.4923
Epoch 10/20, Loss: 2.2602, Acc: 0.5057
Epoch 11/20, Loss: 2.1497, Acc: 0.5108
Epoch 12/20, Loss: 2.0405, Acc: 0.5417
Epoch 13/20, Loss: 2.0163, Acc: 0.5304
Epoch 14/20, Loss: 1.8875, Acc: 0.5767
Epoch 15/20, Loss: 1.8644, Acc: 0.6128
Epoch 16/20, Loss: 1.7737, Acc: 0.6282
Epoch 17/20, Loss: 1.7401, Acc: 0.6303
Epoch 18/20, Loss: 1.6649, Acc: 0.6550
Epoch 19/20, Loss: 1.6387, Acc: 0.6519
Epoch 20/20, Loss: 1.5914, Acc: 0.6684


In [11]:
def test_model(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels.squeeze()).sum().item()
            total += labels.size(0)

    test_loss = running_loss / len(val_loader)
    test_acc = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')


test_model(model, val_loader, criterion)


Test Loss: 2.0673, Test Acc: 0.4444


In [21]:
def save_predictions(model, test_loader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for inputs in test_loader:
            inputs = inputs.to(device)  # Move the inputs to the correct device
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    # Assuming 'sample_submission.csv' is in the required format
    submission = pd.read_csv('sample_submission.csv')
    submission['Label'] = predictions
    submission.to_csv('submission.csv', index=False)

save_predictions(model, test_loader)
